# Project 1 

Importing allowed libraries...

In [32]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## I. Load the Data

In [33]:
# Different paths to access data file

Louise_Path = '/Users/louiseplacidet/Desktop/Machine Learning/Project 1/Git_ML_P1/Data/train.csv'


data_path = Louise_Path #CHANGE HERE BASED ON WHO IS USING THIS

In [34]:
import csv
import numpy as np


def load_csv_data(data_path, sub_sample=False):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int)
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = -1
    
    # sub-sample
    if sub_sample:
        yb = yb[::50]
        input_data = input_data[::50]
        ids = ids[::50]

    return yb, input_data, ids

In [35]:
yb, input_data, ids = load_csv_data(data_path)

In [36]:
input_data

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

In [37]:
yb

array([ 1., -1., -1., ...,  1., -1., -1.])

In [38]:
ids

array([100000, 100001, 100002, ..., 349997, 349998, 349999])

## Build Model

In [39]:
num_samples = len(yb)
tx = np.c_[np.ones(num_samples), input_data]

In [40]:
tx

array([[   1.   ,  138.47 ,   51.655, ...,    1.24 ,   -2.475,  113.497],
       [   1.   ,  160.937,   68.768, ..., -999.   , -999.   ,   46.226],
       [   1.   , -999.   ,  162.172, ..., -999.   , -999.   ,   44.251],
       ...,
       [   1.   ,  105.457,   60.526, ..., -999.   , -999.   ,   41.992],
       [   1.   ,   94.951,   19.362, ..., -999.   , -999.   ,    0.   ],
       [   1.   , -999.   ,   72.756, ..., -999.   , -999.   ,    0.   ]])

# II. Implementation Functions

In [41]:
def compute_loss_mse(y, tx, w):
    """Calculate the loss using mse."""
    e = y - tx.dot(w)
    return 1/2 * np.mean(e**2)

### Least Squares GD

In [42]:
def compute_gradient(y, tx, w):
    """Compute a gradient."""
    e = y - tx.dot(w)
    gradient = - (1/len(e)) * tx.T.dot(e)
    
    return gradient, e

In [43]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    losses = []
    w = initial_w
    for n_iter in range(max_iters):

        gradient, e = compute_gradient(y,tx,w)
        loss = compute_loss(y, tx, w)
        
        w = w - gamma * gradient

    return w, loss

### Least Squares SGD

In [44]:
def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    e = y - tx.dot(w)
    gradient = - (1/len(e)) * tx.T.dot(e)
    return gradient, e

In [45]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


In [46]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """Stochastic gradient descent algorithm."""
    batch_size = 5 ##TBD
    w = initial_w
    
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y,tx,batch_size = batch_size, num_batches = 1):
            gradient , e = compute_stoch_gradient(y_batch,tx_batch, w)
            w = w - gamma*gradient
            loss = compute_loss(y,tx,w)
            print("loss")
            print(loss)
            print("w")
            print(w)
    
    return w, loss

### Least Squares

In [47]:
def least_squares(y, tx):
    
    w = np.linalg.solve(tx.T@tx,tx.T@y)
    loss = compute_loss_mse(y, tx, w)
    
    return w, loss

### Ridge Regression

In [48]:
def ridge_regression(y, tx, lambda_):

    lambdaI = 2* tx.shape[0] * lambda_ * np.eye(tx.shape[1])
    w = np.linalg.solve(tx.T.dot(tx)+lambdaI,tx.T.dot(y))
    loss = compute_loss_mse(y, tx, w)
    
    return w, loss

### Logistic Regression

##### Sigmoid Function

In [49]:
def sigmoid(t):
    """apply the sigmoid function on t."""
    return 1 / (1+np.exp(-t))

##### Calculating Loss for logistic regression

In [50]:
def calculate_loss_logistic(y, tx, w):
    """compute the loss: negative log likelihood."""
    insidesum = np.log(1 + np.exp(tx@w)) - y * (tx@w)
    loss = np.sum(insidesum)
    
    return loss

##### Gradient Descent with Logistic Regression

In [51]:
def calculate_gradient_logistic(y, tx, w):
    """compute the gradient of loss."""
    grad = tx.T @ (sigmoid(tx@w) - y)
    
    return grad

In [52]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    loss = calculate_loss_logistic(y, tx, w)
    
    grad = calculate_gradient_logistic(y, tx, w)
    
    w = w - gamma * grad
    
    return loss, w

In [53]:
def logistic_regression_gradient(y, tx, initial_w, max_iters, gamma):
    #threshold = 1e-8
    
    w = initial_w
    # start the logistic regression
    for i in range(max_iters):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)

        print("Step: "+str(i)+'/'+str(max_iters))
        print("current loss: "+str(loss))
        print("current weights: "+str(w))
        
    return w, loss

##### Newton Method for Logisitic regression

In [70]:
def calculate_hessian(y, tx, w):
    """return the Hessian of the loss function with respect to parameters w."""

    S = np.eye(len(y))*(np.array(sigmoid(tx@w) * (1-sigmoid(tx@w))))
        
    hessian = tx.T @ S @ tx
    return hessian

In [55]:
def logistic_regression_step(y, tx, w):
    """return the loss, gradient, and Hessian."""

    loss = calculate_loss_logistic(y, tx, w)
    gradient = calculate_gradient_logistic(y, tx, w)
    hessian = calculate_hessian(y, tx, w)
    
    return loss, gradient, hessian

In [56]:
def learning_by_newton_method(y, tx, w, gamma):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    # ***************************************************
    # return loss, gradient and Hessian:
    # ***************************************************

    loss, gradient, hessian = logistic_regression_step(y, tx, w)
    
    # ***************************************************
    # update w:
    # ***************************************************
    
    w = w - gamma * np.linalg.inv(hessian) @ gradient
    
    return loss, w

In [57]:
def logistic_regression_newton(y, tx, initial_w, max_iters, gamma):
    #threshold = 1e-8
    w = initial_w
    
    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, gamma)

        # converge criterion
        #losses.append(loss)
        #if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
        #    break
        print("loss: "+ str(loss))
        print("current_w: " + str(w))
        
    return w, loss

##### Regularized Gradient Descent for Logistic Regression

In [58]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and Hessian."""
    loss = calculate_loss_logistic(y, tx, w) + (lambda_ / 2) * np.sum(w**2)
    gradient = calculate_gradient_logistic(y, tx, w) + 2 * lambda_ * w
    #hessian = calculate_hessian(y, tx, w) + 2 * lambda_
    
    return loss, gradient, hessian

In [59]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, gradient, _ = penalized_logistic_regression(y, tx, w, lambda_)
    w = w - gamma * gradient
    
    return loss, w

In [60]:
def reg_logistic_regression_gradient(y, tx, lambda_, initial_w, max_iters, gamma):

    w = initial_w

    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
    
    return w, loss

##### Regularized Newton Method for Logistic Regression

In [61]:
def penalized_logistic_regression_step(y, tx, w,lambda_):
    """return the loss, gradient, and Hessian."""

    loss = calculate_loss_logistic(y, tx, w)(y, tx, w) + (lambda_ / 2) * np.sum(w**2)
    gradient = calculate_gradient_logistic(y, tx, w) + 2 * lambda_ * w
    hessian = calculate_hessian(y, tx, w) + 2 * lambda_
    
    return loss, gradient, hessian

In [62]:
def penalized_learning_by_newton_method(y, tx, w, gamma, lambda_):
    # ***************************************************
    # return loss, gradient and Hessian:
    # ***************************************************

    loss, gradient, hessian = penalized_logistic_regression_step(y, tx, w, lambda_)
    
    # ***************************************************
    # update w:
    # ***************************************************
    
    w = w - gamma * np.linalg.inv(hessian) @ gradient
    
    return loss, w

In [63]:
def reg_logistic_regression_newton(y, tx, initial_w, max_iters, gamma, lambda_):

    w = initial_w
    
    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        loss, w = penalized_learning_by_newton_method(y, tx, w, gamma, lambda_)
        print("loss: "+ str(loss))
        print("current_w: " + str(w))
        
    return w, loss

# III. Implementing the Functions

#### Initial testing hyperparameters

In [71]:
initial_w = np.zeros((tx.shape[1],1))
max_iters = 10
gamma = 0.1
lambda_ = 0.01

In [72]:
w_train, loss = logistic_regression_gradient(yb[:10000], tx[:10000,:], initial_w[:10000], max_iters, gamma)

Step: 0/10
current loss: 69314718.0560066
current weights: [[ 5.00000000e+02 -1.50000000e+03 -1.50000000e+03 ... -1.50000000e+03
  -1.50000000e+03 -1.50000000e+03]
 [-2.26521590e+04  6.79564770e+04  6.79564770e+04 ...  6.79564770e+04
   6.79564770e+04  6.79564770e+04]
 [ 2.45664202e+04 -7.36992604e+04 -7.36992604e+04 ... -7.36992604e+04
  -7.36992604e+04 -7.36992604e+04]
 ...
 [-3.52744190e+05  1.05823257e+06  1.05823257e+06 ...  1.05823257e+06
   1.05823257e+06  1.05823257e+06]
 [-3.52748238e+05  1.05824471e+06  1.05824471e+06 ...  1.05824471e+06
   1.05824471e+06  1.05824471e+06]
 [ 3.65468533e+04 -1.09640560e+05 -1.09640560e+05 ... -1.09640560e+05
  -1.09640560e+05 -1.09640560e+05]]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


Step: 1/10
current loss: inf
current weights: [[ 7.23800000e+02 -2.72380000e+03 -2.72380000e+03 ... -2.72380000e+03
  -2.72380000e+03 -2.72380000e+03]
 [-3.90720670e+03  9.45158427e+04  9.45158427e+04 ...  9.45158427e+04
   9.45158427e+04  9.45158427e+04]
 [ 3.27067438e+04 -1.30972424e+05 -1.30972424e+05 ... -1.30972424e+05
  -1.30972424e+05 -1.30972424e+05]
 ...
 [-3.52729966e+05  1.76370672e+06  1.76370672e+06 ...  1.76370672e+06
   1.76370672e+06  1.76370672e+06]
 [-3.52753733e+05  1.76374668e+06  1.76374668e+06 ...  1.76374668e+06
   1.76374668e+06  1.76374668e+06]
 [ 7.81817453e+04 -2.24369159e+05 -2.24369159e+05 ... -2.24369159e+05
  -2.24369159e+05 -2.24369159e+05]]
Step: 2/10
current loss: inf
current weights: [[ 8.00100000e+02 -3.91120000e+03 -3.91120000e+03 ... -3.91120000e+03
  -3.91120000e+03 -3.91120000e+03]
 [ 1.91709490e+03  1.23413589e+05  1.23413589e+05 ...  1.23413589e+05
   1.23413589e+05  1.23413589e+05]
 [ 3.51166272e+04 -1.86785938e+05 -1.86785938e+05 ... -1.86785

KeyboardInterrupt: 

# IV. Preprocessing the Data

# V. Creating Submission CVS

In [ ]:
def predict_labels(weights, data):
    """Generates class predictions given weights, and a test data matrix"""
    y_pred = np.dot(data, weights)
    y_pred[np.where(y_pred <= 0)] = -1
    y_pred[np.where(y_pred > 0)] = 1
    
    return y_pred

In [ ]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})